In [35]:
import numpy as np 
import pandas as pd
import geopandas as gpd

In [36]:
# load reference data
ref = gpd.read_file(r'ref_data\berlin_ref.gpkg')
#ref = gpd.read_file(r'ref_data\hongkong_ref.gpkg')
#ref = gpd.read_file(r'ref_data\paris_ref.gpkg')
#ref = gpd.read_file(r'ref_data\rome_ref.gpkg')
#ref = gpd.read_file(r'ref_data\saopaulo_ref.gpkg')

In [37]:
ref["id"] = ref.index

In [38]:
def stratified_area_kfold_split(gdf, class_col, folds=5, random_seed=0):
    np.random.seed(random_seed)
    gdf = gdf.copy()
    gdf["area"] = gdf.geometry.area
    gdf["fold"] = -1 
    
    for class_value, class_group in gdf.groupby(class_col):
        class_group = class_group.sample(frac=1, random_state=random_seed)
        total_area = class_group["area"].sum()
        
        fold_areas = [0] * folds
        fold_assignments = [[] for _ in range(folds)]

        for idx, row in class_group.iterrows():
            min_fold = np.argmin(fold_areas)
            fold_areas[min_fold] += row["area"]
            fold_assignments[min_fold].append(idx)

        for fold_num, indices in enumerate(fold_assignments):
            gdf.loc[indices, "fold"] = fold_num

    gdf["gridcode"] = gdf["gridcode"].astype("int32")
    return gdf

In [39]:
split_gdf = stratified_area_kfold_split(ref, class_col='gridcode', folds=5, random_seed=0)

In [42]:
# save split
split_gdf.to_file(r"ref_data\berlin_ref_splitS2S3S4.gpkg", driver="GPKG")
#split_gdf.to_file(r"ref_data\hongkong_ref_splitS2S3S4.gpkg", driver="GPKG")
#split_gdf.to_file(r"ref_data\paris_ref_splitS2S3S4.gpkg", driver="GPKG")
#split_gdf.to_file(r"ref_data\rome_ref_splitS2S3S4.gpkg", driver="GPKG")
#split_gdf.to_file(r"ref_data\saopaulo_ref_splitS2S3S4.gpkg", driver="GPKG")